## DeepFM
DeepFM, as the name suggests, is a deep learning-based extension for Factorization Machines capturing non-linear relationships.

### Architecture
![image](./images/deepfm.webp)

Neural Collaborative Filtering (NCF) utilizes a neural network with two branches that eventually converge to produce a single output. The NCF architecture consists of several key components:

### 1. Input and Output

As conveyed DeepFM is an extension of Factorization Machines, even though the input is quite similar i.e. One-Hot Encoded features. The output is 0–1 giving the probability of whether the user will click or not as sigmoid activation is used in the last layer. However, this can be changed to ReLU or Linear activation for ratings/ranking purposes (explicit feedback).

### 2. Embeddings Layer

This layer is trying to learn embeddings similar to the Latent vectors in the Factorization Machine. So, for each OHE feature, we will learn embeddings of the same size in this layer

From here on, these embeddings will be fed to two different branches

#### a. Factorization Machines
This is a replica of the Factorization Machine where, using the latent vector embeddings we generated in the previous step, will use the below equation

y = w₀ + ∑(wᵢ * xᵢ) + ∑ᵢ(∑ⱼ(<vᵢ . vⱼ> * xᵢ * xⱼ))

Where

    y = Label

    w₀ = Bias

    wᵢ = weights

    xᵢ = Features from One-Hot encoded feature-set

    <vᵢ . vⱼ> = Dot product between latent vectors.

Note: Touch down on this blog to understand the equation

The other branch where the embeddings are fed is
#### b. DNN

This is a general Neural Network (similar to NCF’s MLP) where all these latent vector embeddings are fed to a few hidden layers and forwarded to a final output layer.

### 5. Final Output

The outputs from the Factorization Machine and DNN segments are combined together (concatenated) and then the sigmoid function is applied for the final output. So, a DeepFM can be summarized as

## Implementation

#### 1. Imports

In [1]:
!pip install LibRecommender

In [2]:
import numpy as np
import pandas as pd


from libreco.data import split_by_ratio_chrono, DatasetFeat
from libreco.algorithms import DeepFM

Instructions for updating:
non-resource variables are not supported in the long term


### 2. Dataset
Dummy Data

In [25]:
import numpy as np
import pandas as pd

# Define the number of users and unique items
num_users = 1000
num_items = 20

# Generate random user IDs and item IDs
user_ids = np.arange(1, num_users + 1)
item_ids = np.arange(1, num_items + 1)

# Create random interaction data
data = {
    'user_id': np.random.choice(user_ids, size=num_users * 10),
    'item_id': np.random.choice(item_ids, size=num_users * 10),
}

# Create a pandas DataFrame from the data
df = pd.DataFrame(data).drop_duplicates()

# Display the first few rows of the generated data
print(df.head())

   user_id  item_id
0      463        4
1      138       20
2      406       17
3      314       16
4      230        5


In [26]:
#grouping all interactions by a user as list
df = df.groupby(['user_id'])['item_id'].agg(list).reset_index()

#creating OHE
df['item_id'] = df['item_id'].transform(lambda x: [0 if y+1 not in x else y+1 for y in range(20)])

#save csv
df.to_csv('dummy_data.csv',index=False)

In [28]:
df.head()

,user_id,item_id
0,1,"[0, 0, 0, 4, 0, 0, 0, 8, 9, 10, 0, 0, 0, 0, 0,..."
1,2,"[0, 2, 0, 4, 0, 0, 0, 8, 0, 0, 0, 12, 13, 0, 1..."
2,3,"[0, 0, 0, 0, 0, 0, 7, 0, 9, 0, 11, 0, 13, 0, 0..."
3,4,"[1, 2, 0, 4, 0, 6, 0, 8, 9, 0, 0, 0, 0, 0, 15,..."
4,5,"[1, 0, 3, 0, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 15,..."


In [30]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.4 MB/s eta 0:00:00


In [32]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 8.3 MB/s eta 0:00:00


In [39]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [42]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB

In [43]:
%pip install --upgrade --quiet huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.0/419.0 kB 7.1 MB/s eta 0:00:00


In [48]:
!pip install langchain-huggingface

In [35]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader

## 3.Langchain

In [36]:
#data loader
loader = CSVLoader(file_path="dummy_data.csv")
data = loader.load()

In [37]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [40]:
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [44]:
docsearch = Chroma.from_documents(texts, embeddings)

In [45]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [46]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [49]:
from langchain_huggingface import HuggingFaceEndpoint

In [53]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_length=128,
     max_new_tokens=512,
    temperature=0.5,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [54]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [55]:
qa.run('Suggest 2 articles to user-id 78 using given data which it has not seen.\
Follow this approach 1: Find similar Users and 2: sugest new articles from similar users.\
Also give a reason for suggestion').split('.')

[' To suggest articles to user-id 78 based on the given data, we first need to find similar users and then suggest articles from those similar users that user-id 78 has not seen yet',
 "\n\nFirst, let's find similar users based on the item_id they have interacted with",
 ' We can use the Jaccard similarity coefficient to measure the similarity between two users',
 ' The Jaccard similarity coefficient is the size of the intersection divided by the size of the union of two sets',
 "\n\nHere's the calculation for the Jaccard similarity coefficient between user-id 78 and each of the given users:\n\n1",
 ' User-id 672: Jaccard similarity = (len(set(item_id[672]) & set(item_id[78])) / len(set(item_id[672]) | set(item_id[78]))) = 0',
 '2222\n2',
 ' User-id 719: Jaccard similarity = (len(set(item_id[719]) & set(item_id[78])) / len(set(item_id[719]) | set(item_id[78]))) = 0',
 '1667\n3',
 ' User-id 697: Jaccard similarity = (len(set(item_id[697]) & set(item_id[78])) / len(set(item_id[697]) | se

In [56]:
qa.run('Suggest 2 articles to user-id 78 using given data which it has not seen.').split('.')

[' Based on the given data, user-id 78 has not interacted with items 2, 3, 13, 14, 16, 17',
 ' Therefore, two articles that could be suggested to user-id 78 based on the given data are items with ids 2 and 13',
 " However, it's important to note that this suggestion is based solely on the given data and does not take into account other factors such as the user's interests or preferences",
 '']